# Produce Plots for Publication

Use self built surface classification library `surface_classify` available at https://github.com/FelixWodaczek/surface_classify.git.

## Imports

In [ ]:
import sys
sys.path.append("../py_src")

from glob import glob
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from matplotlib.ticker import IndexLocator, MaxNLocator
from matplotlib import colors

import sort_neigh

from ase.io import read as ase_read
from ase.neighborlist import natural_cutoffs, NeighborList
from dscribe.descriptors import LMBTR, SOAP

## Standard Values

In [ ]:
simu_type = 'rh'
target_dir = "../test_data/230118_finalres_newlocalstruct_preclass/"
pd_target_dir = target_dir+'pd'
target_dir = target_dir + simu_type
n_particles = 1577
n_rhod = 15


rcut=5.2
nmax=4
lmax=3
sigma=1.
gamma_kernel=1.

## Figure 1, Available Sites on Nanoparticle

Analyse the available sites in a nanoparticle containing only Rh.

### Define Target

In [ ]:
only_cu_dir = target_dir + "/cunanoparticle"
only_cu_path = only_cu_dir + "/cusingle.lammpstrj"

In [ ]:
rh_classifier = sort_neigh.NeighbourClassifier(
    local_structures_path=os.path.abspath("../src/localstructures_final_mc"),
    non_class_max=14
)
rh_classifier.load_identifiers(
    rcut=rcut, nmax=nmax, lmax=lmax, 
    sigma=sigma, gamma_kernel=gamma_kernel,
)

### Run Analysis via Classifier
Use the surface classifier to determine what sites are identified at which position.

In [ ]:
full_particle = ase_read(only_cu_path)
at_pos = full_particle.get_positions()

In [ ]:
from ase.visualize import view
mode="class_all"

cut_off = natural_cutoffs(full_particle, mult=.98)# mult=0.98)
neighbour_list = NeighborList(cut_off, bothways=True, self_interaction=False)
neighbour_list.update(full_particle)

cu_cat_counter = np.zeros(shape=(rh_classifier.n_classes), dtype=np.int32)
categories = np.zeros((len(full_particle),), dtype=np.int32)
neighbours = np.zeros((len(full_particle),), dtype=np.int32)

ind_soaps = np.zeros((len(full_particle), rh_classifier.descr.get_number_of_features()))
for index in range(len(full_particle)):
    neighbour_indices, trash = neighbour_list.get_neighbors(index)
    neighbour_indices = np.append(np.array([index]), neighbour_indices, axis=0)
    neighbour_particle = full_particle[neighbour_indices]
    
    # Make center atom Rh
    neighbour_particle.symbols[:] = 'Cu'
    neighbour_particle.symbols[0] = 'Rh'
    
    ind_soaps[index] = rh_classifier.descr.create(neighbour_particle, positions=[0])
    n_neigh, class_id = rh_classifier.classify(neighbour_particle, mode=mode, ensure_position=False)

    cu_cat_counter[class_id] += 1
    neighbours[index] = int(n_neigh)
    categories[index] = int(class_id)
    if int(class_id) == 8:
        print(neighbour_particle)
        print(neighbour_particle.get_chemical_symbols())

In [ ]:
for ii in range(len(cu_cat_counter)):
    print(rh_classifier.id_to_cat(ii), ': %u'%cu_cat_counter[ii])

### Unsupervised Machine Learning
Use unsupervised ML to perform the same task and compare results afterwards

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans, DBSCAN, Birch
from dscribe.descriptors import SOAP

ml_classifier = sort_neigh.USMLClassifier()

train_on_particle = False
if train_on_particle:
    n_clust = ml_classifier.train_on_particle(
        full_particle,
        soap_species=["Cu"], dim_red=PCA(n_components=4), 
        clusterer=Birch(n_clusters=10),
        rcut=rcut, nmax=nmax, lmax=lmax, sigma=sigma
    )
else:
    n_clust = ml_classifier._train_on_data(
        ind_soaps,
        dim_red=PCA(n_components=20), clusterer=Birch(n_clusters=8)
    )
    ml_classifier.descr = rh_classifier.descr

# soaps = ml_classifier.descr.create(full_particle)
reduced = ml_classifier.dim_red.transform(ind_soaps)

### Plotting
Create view of nanoparticle and reduced mapping for Paper

In [ ]:
### Renaming Dict for better labels:
pretty_label = True
if pretty_label:
    prettylabel_dict = {
            "12": "12 neighbours",
            "11": "11 neighbours",
            "3_111_ad_atom": "111 adatom",
            "4_111_ad_atom_pair": "111 adatom_pair",
            "4_100_ad_atom": "100 adatom",
            "5_211_ad_atom": "211 adatom" ,
            "5_110_ad_atom": "110 adatom",
            "5_111_terrace": "111 terrace",
            "6_100-110_interface": "100-110 interface",
            "6_100_terrace": "100 terrace",
            "7_110": "110",
            "7_211": "211",
            "8_100": "100",
            "8_111_vacant_site": "111 vacancy",
            "9_111": "111",
            "9_invalid": "9 subsurface",
            "9_invalid_2": "9 subsurface",
    }

In [ ]:
def norm_categories(a_categories, labels=None):
    c_range = np.arange(np.max(a_categories), np.min(a_categories)-2, -1)
    if labels is not None and len(c_range)-1 != len(labels):
        raise ValueError("Range of argument 'counter' needs to be same length as argument 'labels' if given.")
        
    ret_cats = a_categories.copy()
    
    if labels is not None:
        ret_labels = labels.copy()
    
    reduce_at_this = False
    for value_index, value in enumerate(c_range):
        reduce_at_next = value not in ret_cats
        
        if reduce_at_this:
            largereq_than_last = ret_cats > value
            ret_cats[largereq_than_last] -= 1
            if labels is not None:
                ret_labels.pop(len(c_range)-value_index-1)

        reduce_at_this = reduce_at_next
        
    ret_cats -= np.min(ret_cats)
    return ret_cats, ret_labels

        
def cmap_from_categories(colors, or_map_name="tab20", n_orcolors=20):
    c_range = np.max(colors)-np.min(colors)+1
    normed_colors = (colors-np.min(colors))/(c_range-1)
    normed_colors *= (c_range-1)/(c_range)
    normed_colors += 1./(2*c_range)

    c_ticks = np.linspace(0, 1, c_range*2+1, endpoint=True)[1::2]

    tab20 = cm.get_cmap(or_map_name, 256)
    color_range = np.linspace(0, c_range/float(n_orcolors), 500, endpoint=False)
    cmap = ListedColormap(tab20(color_range))
    
    return normed_colors, cmap, c_ticks


In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap

%matplotlib auto
plt.ion()

cond = np.s_[...] # neighbours < 17

colors = categories[cond].copy()
# colors = n_clust

c_labels = np.arange(np.min(colors), np.max(colors)+1)
c_labels = [rh_classifier.id_to_cat(c_label) for c_label in c_labels]

order_by_neighbors = True
if order_by_neighbors:
    colors[colors==np.min(colors)] = np.max(colors) + 1
    colors[colors==np.min(colors)] = np.max(colors) + 1
    c_labels.append(c_labels[0])
    c_labels.append(c_labels[1])
    c_labels.pop(0)
    c_labels.pop(0)
    
    while(len(c_labels) != np.max(colors) + 1 - np.min(colors)):
          c_labels.pop(0)
    
cont_values, c_labels = norm_categories(colors, labels=c_labels)
cat_clabels = c_labels.copy()

cat_normed_colors, cat_cmap, cat_cticks = cmap_from_categories(cont_values, or_map_name='tab10', n_orcolors=10)

label_color_dict = {c_label: color for c_label, color in zip(c_labels, cat_cticks)}

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

sc = ax.scatter(
    at_pos[cond, 0], at_pos[cond, 1], at_pos[cond, 2], c=cat_normed_colors, cmap=cat_cmap, alpha=1,
    s=800, edgecolors="k", vmin=0, vmax=1
)

if pretty_label:
    plot_labels = [
        prettylabel_dict[c_label] if c_label in prettylabel_dict.keys() else c_label for c_label in c_labels
    ]
else:
    plot_labels = c_labels
    
ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

bar = fig.colorbar(sc)
bar.set_ticks(cat_cticks)
bar.set_ticklabels(plot_labels)
ax.set_axis_off()

plt.tight_layout()
fig.savefig("threed_particle.pdf", format='pdf')
plt.show()

In [ ]:
soaps_from_classifier = []
labels = []

for key in rh_classifier.identification_dict.keys():
    entry = rh_classifier.identification_dict[key]
    if entry is not None:
        soaps_from_classifier.append(entry["soap_descr"][:, 0, :])
        labels.append(entry["id"])

buff = soaps_from_classifier[0].copy()
for ii_soap in range(1, len(soaps_from_classifier)):
    buff = np.append(buff, soaps_from_classifier[ii_soap], axis=0)

soaps_from_classifier = buff.copy()
del buff

buff = []
for label in labels:
    for entry in label:
        buff.append(entry)

labels=buff
del buff

if pretty_label:
    labels = [
        prettylabel_dict[blabel] if blabel in prettylabel_dict.keys() else blabel for blabel in labels
    ]
print(labels)
print(soaps_from_classifier.shape)
soap_prediction = ml_classifier.dim_red.transform(soaps_from_classifier)

In [ ]:
compare_soaps = True

%matplotlib auto
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.set_title("PCA Map of Unsupervised Regression")
c_labels = np.arange(np.min(n_clust), np.max(n_clust)+1)
normed_colors, cmap, c_ticks = cmap_from_categories(n_clust)

sc = ax.scatter(reduced[:, 0], reduced[:, 1], c=normed_colors, cmap=cmap, vmin=0, vmax=1)

if compare_soaps:
    ax.scatter(soap_prediction[:, 0], soap_prediction[:, 1], c="k")

    for ii_label, label in enumerate(labels):  
        ax.annotate(label, soap_prediction[ii_label, 0:2])
    
cbar = fig.colorbar(sc)
cbar.set_ticks(c_ticks)
cbar.set_ticklabels(c_labels)
plt.tight_layout()
fig.savefig("unsupervised_ml_pca.pdf", format='pdf')
plt.show()

fontsize = 15
ms = 120
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
ax.set_title("PCA Map of Classification by Similarity", fontsize=fontsize)
sc = ax.scatter(
    reduced[:, 0], reduced[:, 1], 
    c=cat_normed_colors, s=ms, cmap=cat_cmap, vmin=0, vmax=1
)

if compare_soaps:
    ax.scatter(soap_prediction[:, 0], soap_prediction[:, 1], c="k", s=ms)
    for ii_label, label in enumerate(labels):
        ax.text(
            soap_prediction[ii_label, 0]+0.5, soap_prediction[ii_label, 1]-0.015, 
            label, ha='left', fontsize=fontsize-2
        )
    
cbar = fig.colorbar(sc) 

cbar.set_ticks(cat_cticks)
cbar.set_ticklabels(plot_labels, fontsize=fontsize-2)

# ax.set_xlim([-7, 21])

ax.tick_params(axis='x', labelsize=fontsize-2)
ax.tick_params(axis='y', labelsize=fontsize-2)
ax.set_xlabel('Principal Axis 1', fontsize=fontsize)
ax.set_ylabel('Principal Axis 2', fontsize=fontsize)

plt.tight_layout()
fig.savefig("classification_pca.pdf", format='pdf')
plt.show()

## Figure 2, Rh Positions over Time
After defining all the available positions in the nanoparticle, analyse where the Rh sit in each timestep.

### Run Evaluation on Files
Set `newcats` to `True` to rerun evaluation, otherwise pre-existing evaluation is loaded from .txt files.

In [ ]:
%matplotlib inline

def load_all(target_dir):
    target_folders = [
        target_dir+"/mc",
        target_dir+"/mcmd"
    ]

    results_dict = {}
    n_cats = 0

    bulkify = True # Sum up 11 and 12 neighbours to "bulk"

    for target_folder in target_folders:
        for target_file in glob(target_folder+"/*.lammpstrj"):
            target_file = os.path.abspath(target_file)
            only_file = os.path.basename(target_file).split(".")[0]
            out_dir = os.path.join(os.path.dirname(target_file), only_file+"_soap_sorted_counts.txt")
            save_txt_path = os.path.join(os.path.dirname(target_file), only_file+"_soap_sorted_counts.txt")
            
            dir_name = save_txt_path.split("/")[-2]
            cur_key = '_'.join([dir_name, only_file])
            results_dict[cur_key] = {}
            
            sorted_counts, timesteps, sorted_cats = sort_neigh.NeighbourSort.load_sort_cat(save_txt_path)
            
            # print(dir_name)
            # sort_neigh.NeighbourSort.plot_dist(sorted_cats, sorted_counts)
            
            if bulkify:
                bulk_args = []
                for ii_scat, scat in enumerate(sorted_cats):
                    try:
                        if int(scat) >= 10 and int(scat) < 99:
                            bulk_args.append(ii_scat)
                    except:
                        pass
                bulk_args = np.array(bulk_args)
                bulk_slicer = np.s_[:, bulk_args]
                sorted_counts = np.append(
                    sorted_counts, np.sum(
                        sorted_counts[bulk_slicer], axis=-1
                    )[:, np.newaxis], axis=-1
                )
                sorted_cats.append('bulk')
                sorted_counts[bulk_slicer] = 0

            results_dict[cur_key]["sorted_counts"] = sorted_counts
            results_dict[cur_key]["timesteps"] = timesteps
            results_dict[cur_key]["sorted_cats"] = sorted_cats
            n_cats = max(n_cats, len(sorted_cats))
    
    return results_dict, n_cats

results_dict, n_cats = load_all(target_dir=target_dir)
pd_results_dict, pd_ncats = load_all(target_dir=pd_target_dir)

### Plot Raw Results

In [ ]:
def reshape_res_dict(results_dict):
    block_size = 1000
    md_counts = np.zeros((3 , n_cats), dtype=np.float64)
    md_errs = md_counts.copy()
    mcmd_counts = np.zeros((3 , n_cats), dtype=np.float64)
    mcmd_errs = mcmd_counts.copy()

    counter = 0
    for key, subdict in sorted(results_dict.items()):
        if not counter: # For first dictionary
            sorted_cats = subdict["sorted_cats"]
        if not "mcmd" in key:
            md_counts[counter%3, :] = np.sum(subdict["sorted_counts"], axis=0)
            md_counts[counter%3, :] = md_counts[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            md_counts[counter%3, :] *= 100

            md_errs[counter%3, :] = sort_neigh.NeighbourSort.block_average(subdict["sorted_counts"], block_size=block_size)        
            md_errs[counter%3, :] = md_errs[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            md_errs[counter%3, :] *= 100
        else:
            mcmd_counts[counter%3, :] = np.sum(subdict["sorted_counts"], axis=0)
            mcmd_counts[counter%3, :] = mcmd_counts[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            mcmd_counts[counter%3, :] *= 100
            
            mcmd_errs[counter%3, :] = sort_neigh.NeighbourSort.block_average(subdict["sorted_counts"], block_size=block_size)        
            mcmd_errs[counter%3, :] = mcmd_errs[counter%3, :] / (subdict["sorted_counts"].shape[0] * n_rhod)
            mcmd_errs[counter%3, :] *= 100
        counter+=1

    return md_counts, md_errs, mcmd_counts, mcmd_errs, sorted_cats

md_counts, md_errs, mcmd_counts, mcmd_errs, sorted_cats = reshape_res_dict(results_dict=results_dict)
md_notzero = np.logical_not(np.logical_and.reduce(md_counts==0, axis=0))
mcmd_notzero = np.logical_not(np.logical_and.reduce(mcmd_counts==0, axis=0))

pd_md_counts, pd_md_errs, pd_mcmd_counts, pd_mcmd_errs, pd_sorted_cats = reshape_res_dict(results_dict=pd_results_dict)
pd_md_notzero = np.logical_not(np.logical_and.reduce(pd_md_counts==0, axis=0))
pd_mcmd_notzero = np.logical_not(np.logical_and.reduce(pd_mcmd_counts==0, axis=0))

## Plot Total Results

### Barplot

In [ ]:
fontsize=20

temperatures = ['400', '500', '600']

fig, axis = plt.subplots(figsize=(12,8))
widths = np.array([-0.3, 0, 0.3])
alphas = [0.2, 0.5, 1]

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if md_notzero[ii_cat]:
        draw_cats.append(cat)

total_nonzero = np.sum(md_notzero.astype(np.int8))
x_locs = np.arange(total_nonzero)
draw_bars = md_counts[:, md_notzero]

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(cat_cmap(label_color_dict[draw_cat]))
    except:
        draw_colors.append('gray')


legend_patches = []
for ii_bars in range(md_counts.shape[0]):
    # Draw categories that are on nanoparticle in colour
    axis.bar(
        x_locs+widths[ii_bars], draw_bars[ii_bars, :], width=0.3,
        color=draw_colors, align='center', alpha=alphas[ii_bars]
    )
    
    # Make annotations with values
    annotation_cond = draw_bars[ii_bars, :] != 0
    annotations = draw_bars[ii_bars, :][annotation_cond]
    for ll_ann, annotation in enumerate(annotations):
        axis.annotate(
            "%.4f"%annotation,
            [x_locs[annotation_cond][ll_ann]+widths[ii_bars], annotation*1.1], 
            ha='center', rotation=45, fontsize=fontsize-10
        )

    cur_patch = patches.Patch(
        color='k', label=temperatures[ii_bars]+" K", alpha=alphas[ii_bars]
    )
    legend_patches.append(cur_patch)

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

axis.set_yscale("log")
axis.set_xticks(x_locs, plot_labels, fontsize=fontsize-7)
axis.tick_params(axis='y', labelsize=fontsize-7)

axis.set_xlabel('site type', fontsize=fontsize)
axis.set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)
axis.set_ylim([axis.get_ylim()[0], 400])
# axis.autoscale(tight=True)
axis.legend(handles=legend_patches, fontsize=fontsize, loc='upper left')

plt.tight_layout()
fig.savefig("barplot_mc.pdf", format="pdf")
plt.show()

In [ ]:
fontsize=20

temperatures = ['400', '500', '600']

fig, axis = plt.subplots(figsize=(12,8))
widths = np.array([-0.3, 0, 0.3])
alphas = [0.2, 0.5, 1]

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if mcmd_notzero[ii_cat]:
        draw_cats.append(cat)

total_nonzero = np.sum(mcmd_notzero.astype(np.int8))
x_locs = np.arange(total_nonzero)
draw_bars = mcmd_counts[:, mcmd_notzero]

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(cat_cmap(label_color_dict[draw_cat]))
    except:
        draw_colors.append('gray') 


legend_patches = []
for ii_bars in range(mcmd_counts.shape[0]):
    # Draw categories that are on nanoparticle in colour
    axis.bar(
        x_locs+widths[ii_bars], draw_bars[ii_bars, :], width=0.3,
        color=draw_colors, align='center', alpha=alphas[ii_bars]
    )
    
    # Make annotations with values
    annotation_cond = draw_bars[ii_bars, :] != 0
    annotations = draw_bars[ii_bars, :][annotation_cond]
    for ll_ann, annotation in enumerate(annotations):
        axis.annotate(
            "%.4f"%annotation,
            [x_locs[annotation_cond][ll_ann]+widths[ii_bars], annotation*1.1], 
            ha='center', rotation=45, fontsize=fontsize-10
        )

    cur_patch = patches.Patch(
        color='k', label=temperatures[ii_bars]+" K", alpha=alphas[ii_bars]
    )
    legend_patches.append(cur_patch)

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

axis.set_yscale("log")
axis.set_xticks(x_locs, plot_labels, fontsize=fontsize-7)
axis.tick_params(axis='y', labelsize=fontsize-7)

axis.set_xlabel('site type', fontsize=fontsize)
axis.set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)
axis.set_ylim([axis.get_ylim()[0], 400])
# axis.autoscale(tight=True)
axis.legend(handles=legend_patches, fontsize=fontsize, loc='upper left')

plt.tight_layout()
fig.savefig("barplot_mcmd.pdf", format="pdf")
plt.show()

### Broken Axis

In [ ]:
markers = ['o', 'v', 's', '*', 'X', 'D', 'p', 'h']
fontsize = 15

md_lines = md_counts[:, mcmd_notzero]
md_draw_errs = md_errs[:, mcmd_notzero]
total_nonzero = np.sum(md_notzero.astype(np.int8))
x_range = np.arange(3)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6,5))
fig.subplots_adjust(hspace=0.05)

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if md_notzero[ii_cat]:
        draw_cats.append(cat)

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(cat_cmap(label_color_dict[draw_cat]))
    except:
        draw_colors.append('gray')

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

for axis in axes:
    for ii_line in range(-1, -total_nonzero-1, -1):
        axis.errorbar(
            x_range, md_lines[:, ii_line], yerr=md_draw_errs[:, ii_line],
            color=draw_colors[ii_line], marker=markers[ii_line],
            label=plot_labels[ii_line]    
        )
        axis.fill_between(
            x_range, 
            md_lines[:, ii_line]-md_draw_errs[:, ii_line], 
            md_lines[:, ii_line]+md_draw_errs[:, ii_line],
            color=draw_colors[ii_line],
            alpha=0.5
        )

axes[0].set_ylim([90, 100])
axes[1].set_ylim([2, 5])
axes[2].set_ylim([-0.015, 0.065])

# Format ylabels
offset = axes[0].get_ylim()[0]+(0.1*axes[0].get_ylim()[0])
axes[0].yaxis.set_major_locator(MaxNLocator(3))
axes[1].yaxis.set_major_locator(MaxNLocator(2))
axes[2].yaxis.set_major_locator(MaxNLocator(3))
axes[0].minorticks_on()
axes[1].minorticks_on()
axes[2].minorticks_on()

# Interruption
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
axes[0].plot([0, 1], [0, 0], transform=axes[0].transAxes, **kwargs)
axes[1].plot([0, 1], [1, 1], transform=axes[1].transAxes, **kwargs)
axes[1].plot([0, 1], [0, 0], transform=axes[1].transAxes, **kwargs)
axes[2].plot([0, 1], [1, 1], transform=axes[2].transAxes, **kwargs)

# set xlabels
axes[0].spines.bottom.set_visible(False)
axes[1].spines.top.set_visible(False)
axes[1].spines.bottom.set_visible(False)
axes[2].spines.top.set_visible(False)
axes[0].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[1].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].set_xticks(x_range)
axes[2].set_xticklabels(temperatures, fontsize=fontsize*0.8)

# axes[0].grid()
# axes[1].grid()
# axes[2].grid()

# axis.set_yscale('log')

axes[0].set_title("Title Suggestions Welcome, mc %s"%simu_type, fontsize=fontsize)
axes[1].legend(loc=[0.08, 0.6], fontsize=fontsize*0.8)
axes[2].set_xlabel('Temperature [K]', fontsize=fontsize)
axes[1].set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)

fig.savefig("lines_brokenax_mc_%s.pdf"%simu_type, format='pdf')
plt.show()

In [ ]:
markers = ['o', 'v', 's', '*', 'X', 'D', 'p', 'h']
fontsize = 15

mcmd_lines = mcmd_counts[:, mcmd_notzero]
mcmd_draw_errs = mcmd_errs[:, mcmd_notzero]
total_nonzero = np.sum(mcmd_notzero.astype(np.int8))
x_range = np.arange(3)

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6,5))
fig.subplots_adjust(hspace=0.05)

draw_cats = []
for ii_cat, cat in enumerate(sorted_cats):
    if mcmd_notzero[ii_cat]:
        draw_cats.append(cat)

draw_colors = []
for draw_cat in draw_cats:
    try:
        draw_colors.append(cat_cmap(label_color_dict[draw_cat]))
    except:
        draw_colors.append('gray')

if pretty_label:
    plot_labels = [
        prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
    ]
else:
    plot_labels = draw_cats

for axis in axes:
    for ii_line in range(-1, -total_nonzero-1, -1):
        axis.errorbar(
            x_range, mcmd_lines[:, ii_line], yerr=mcmd_draw_errs[:, ii_line],
            color=draw_colors[ii_line], marker=markers[ii_line],
            label=plot_labels[ii_line]    
        )
        axis.fill_between(
            x_range, 
            mcmd_lines[:, ii_line]-mcmd_draw_errs[:, ii_line], 
            mcmd_lines[:, ii_line]+mcmd_draw_errs[:, ii_line],
            color=draw_colors[ii_line],
            alpha=0.5
        )

axes[0].set_ylim([96.4, 100])
axes[1].set_ylim([0.46, 1.53])
axes[2].set_ylim([-0.015, 0.038])

# Format ylabels
offset = axes[0].get_ylim()[0]+(0.1*axes[0].get_ylim()[0])
axes[0].yaxis.set_major_locator(MaxNLocator(3))
axes[1].yaxis.set_major_locator(MaxNLocator(2))
axes[2].yaxis.set_major_locator(MaxNLocator(3))
axes[0].minorticks_on()
axes[1].minorticks_on()
axes[2].minorticks_on()

# Interruption
d = .5
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
axes[0].plot([0, 1], [0, 0], transform=axes[0].transAxes, **kwargs)
axes[1].plot([0, 1], [1, 1], transform=axes[1].transAxes, **kwargs)
axes[1].plot([0, 1], [0, 0], transform=axes[1].transAxes, **kwargs)
axes[2].plot([0, 1], [1, 1], transform=axes[2].transAxes, **kwargs)

# set xlabels
axes[0].spines.bottom.set_visible(False)
axes[1].spines.top.set_visible(False)
axes[1].spines.bottom.set_visible(False)
axes[2].spines.top.set_visible(False)
axes[0].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[1].tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False,
    labelsize=fontsize*0.6
)
axes[2].set_xticks(x_range)
axes[2].set_xticklabels(temperatures, fontsize=fontsize*0.8)

# axes[0].grid()
# axes[1].grid()
# axes[2].grid()

# axis.set_yscale('log')

axes[0].set_title("Title Suggestions Welcome, mcmd %s"%simu_type, fontsize=fontsize)
axes[1].legend(loc=[0.08, 0.6], fontsize=fontsize*0.8)
axes[2].set_xlabel('Temperature [K]', fontsize=fontsize)
axes[1].set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)

fig.savefig("lines_brokenax_mcmd_%s.pdf"%simu_type, format='pdf')
plt.show()

### Logplot All Data

In [ ]:
def draw_rhpd(axis, rh_lines, rh_errs, pd_lines, pd_errs, all_not_zero, **kwargs):
    markers = kwargs.get('markers', ['o', 'v', 's', '*', 'X', 'D', 'p', 'h'])
    fontsize = kwargs.get('fontsize', 15)
    ms = kwargs.get('ms', 10)
    lw = kwargs.get('lw', 3)

    total_nonzero = np.sum(all_not_zero.astype(np.int8))
    x_range = np.arange(3)

    draw_cats = []
    for ii_cat, cat in enumerate(sorted_cats):
        if all_not_zero[ii_cat]:
            draw_cats.append(cat)

    draw_colors = []
    for draw_cat in draw_cats:
        try:
            draw_colors.append(cat_cmap(label_color_dict[draw_cat]))
        except:
            draw_colors.append('gray')

    if pretty_label:
        plot_labels = [
            prettylabel_dict[nz_cat] if nz_cat in prettylabel_dict.keys() else nz_cat for nz_cat in draw_cats
        ]
    else:
        plot_labels = draw_cats

    for ii_line in range(-1, -total_nonzero-1, -1):
        nz_tf = md_lines[:, ii_line] != 0
        axis.errorbar(
            x_range[nz_tf], rh_lines[:, ii_line][nz_tf], yerr=rh_errs[:, ii_line][nz_tf],
            color=draw_colors[ii_line], marker=markers[ii_line%len(markers)],
            markersize=ms, linewidth=lw,
            label=plot_labels[ii_line]
        )
        nz_tf = mcmd_lines[:, ii_line] != 0
        axis.errorbar(
            x_range[nz_tf], pd_lines[:, ii_line][nz_tf], yerr=pd_errs[:, ii_line][nz_tf],
            color=draw_colors[ii_line], marker=markers[ii_line%len(markers)], 
            linestyle='--', markersize=ms, linewidth=lw,
            label=plot_labels[ii_line]    
        )
    axis.set_yscale('log')

    axis.set_xticks(x_range)
    axis.set_xticklabels(temperatures, fontsize=fontsize*0.8)

    # axis.set_ylim((1e0, 1.3e2))

    axis.set_title(kwargs.get('title', 'nd'), fontsize=fontsize)
    # axis.legend(loc=[0.08, 0.6], fontsize=fontsize*0.8)
    axis.set_xlabel('Temperature [K]', fontsize=fontsize)
    # axis.set_ylabel('Rh atoms per surface type [%]', fontsize=fontsize)
    return axis

In [ ]:
from matplotlib.ticker import IndexLocator, MaxNLocator
from matplotlib import colors

all_not_zero = np.logical_or(pd_md_notzero, md_notzero)

md_lines = md_counts[:, all_not_zero]
md_draw_errs = md_errs[:, all_not_zero]
mcmd_lines = pd_md_counts[:, all_not_zero]
mcmd_draw_errs = pd_md_errs[:, all_not_zero]

mcmd_all_not_zero = np.logical_or(pd_mcmd_notzero, pd_md_notzero)

md_lines = mcmd_counts[:, mcmd_all_not_zero]
md_draw_errs = mcmd_errs[:, mcmd_all_not_zero]
mcmd_lines = pd_mcmd_counts[:, mcmd_all_not_zero]
mcmd_draw_errs = pd_mcmd_errs[:, mcmd_all_not_zero]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 6))
draw_rhpd(
    axes[0], 
    md_counts[:, all_not_zero], md_errs[:, all_not_zero],
    pd_md_counts[:, all_not_zero], pd_md_errs[:, all_not_zero],
    all_not_zero,
    title='md'
)
draw_rhpd(
    axes[1], 
    mcmd_counts[:, mcmd_all_not_zero], mcmd_errs[:, mcmd_all_not_zero],
    pd_mcmd_counts[:, mcmd_all_not_zero], pd_mcmd_errs[:, mcmd_all_not_zero],
    mcmd_all_not_zero,
    title='mcmd'
)


plt.tight_layout()
fig.savefig("lines_allog_mcmd.pdf", format='pdf')
plt.show()

